🔧 **Setup Required**: Before running this notebook, please follow the [setup instructions](../README.md#setup-instructions) to configure your environment and API keys. **You will need WANDB_API_KEY environment variable set and Elasticsearch running with indexed data**

# RAG Evaluation with Weights & Biases Experiment Tracking 📊🔬

## 📋 Overview

This notebook demonstrates how to integrate **Weights & Biases (W&B) experiment tracking** with our RAG evaluation pipeline. By combining our custom evaluation components with W&B's powerful experiment tracking capabilities, we can:

1. **Track evaluation metrics** across different RAG configurations
2. **Compare experiments** systematically with visual dashboards
3. **Monitor pipeline execution** with detailed tracing
4. **Store and version** evaluation datasets and results
5. **Share results** with team members through W&B's collaborative platform

## 🎯 Learning Objectives

By the end of this notebook, you will understand how to:
- Set up Weights & Biases tracking for Haystack pipelines
- Log RAGAS evaluation metrics to W&B experiments
- Create comparative experiments between different RAG systems
- Use W&B's WeaveConnector for detailed pipeline tracing
- Organize and analyze experiment results in W&B dashboards

## 🏗️ Architecture Overview

```
CSV Data → RAGDataAugmenter → RagasEvaluation → W&B Logging
    ↑              ↑                 ↑              ↑
CSVReader    SuperComponent    RAGAS Framework  WeaveConnector
                                                      ↓
                                              W&B Dashboard
```

**Key Benefits:**
- **Experiment Tracking**: Systematic comparison of RAG configurations
- **Reproducibility**: Version control for datasets, code, and results
- **Collaboration**: Share insights with team through W&B platform
- **Visualization**: Rich dashboards for metric analysis and trend monitoring

---

## 🔧 Setup: Environment and Dependencies

First, let's set up the necessary environment variables and imports for W&B integration.

In [1]:
import os
import wandb
from pathlib import Path

# Enable Haystack tracing for W&B integration
os.environ["HAYSTACK_CONTENT_TRACING_ENABLED"] = "true"

# Verify W&B API key is set
if not os.getenv("WANDB_API_KEY"):
    print("⚠️  WANDB_API_KEY not found in environment variables.")
    print("Please set your W&B API key: export WANDB_API_KEY=your_key_here")
    print("You can find your API key at: https://wandb.ai/authorize")
else:
    print("✅ W&B API key found in environment")

# Verify OpenAI API key for evaluation
if not os.getenv("OPENAI_API_KEY"):
    print("⚠️  OPENAI_API_KEY not found in environment variables.")
else:
    print("✅ OpenAI API key found in environment")

✅ W&B API key found in environment
✅ OpenAI API key found in environment


## 📦 Import Custom Components and Dependencies

Now let's import our custom RAG evaluation components and the W&B integration components.

In [2]:
# Import our custom evaluation components
from scripts.ragasevaluation import (
    CSVReaderComponent,
    RAGDataAugmenterComponent, 
    RagasEvaluationComponent
)

# Import RAG SuperComponents for testing
from scripts.rag.naiverag import naive_rag_sc
from scripts.rag.hybridrag import hybrid_rag_sc

# Import Haystack pipeline components
from haystack import Pipeline

# Import W&B WeaveConnector for tracing
from haystack_integrations.components.connectors.weave import WeaveConnector

# Import RAGAS metrics
from ragas.metrics import (
    LLMContextRecall, 
    Faithfulness, 
    FactualCorrectness, 
    ResponseRelevancy, 
    ContextEntityRecall, 
    NoiseSensitivity
)

import pandas as pd
from datetime import datetime
import json

print("✅ All components imported successfully")

/Users/laurafunderburk/Documents/GitHub/Building-Natural-Language-Pipelines/ch6/.venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


✅ All components imported successfully


## 🧪 Enhanced Evaluation Component with W&B Logging

Let's create an enhanced version of our evaluation pipeline that integrates seamlessly with Weights & Biases experiment tracking.

In [3]:
class RAGEvaluationExperiment:
    """
    Enhanced RAG evaluation workflow with Weights & Biases integration.
    
    This class wraps our custom components and provides W&B experiment tracking,
    including metric logging, artifact storage, and pipeline tracing.
    """
    
    def __init__(self, 
                 project_name: str = "rag-evaluation-experiments",
                 experiment_name: str = None):
        """
        Initialize the RAG evaluation experiment with W&B tracking.
        
        Args:
            project_name: W&B project name for organizing experiments
            experiment_name: Specific experiment run name (auto-generated if None)
        """
        self.project_name = project_name
        self.experiment_name = experiment_name or f"rag-eval-{datetime.now().strftime('%Y%m%d-%H%M%S')}"
        
        # Initialize W&B run
        self.run = None
        
        # Pipeline components (will be initialized in setup_pipeline)
        self.pipeline = None
    
    def setup_pipeline(self, 
                      rag_supercomponent, 
                      metrics_list: list,
                      config: dict = None):
        """
        Set up the evaluation pipeline with W&B tracking.
        
        Args:
            rag_supercomponent: The RAG SuperComponent to evaluate
            metrics_list: List of RAGAS metrics to compute
            config: Additional configuration parameters to log
        """
        
        # Initialize W&B run with configuration
        self.run = wandb.init(
            project=self.project_name,
            name=self.experiment_name,
            config={
                "rag_system": rag_supercomponent.__class__.__name__,
                "metrics": [metric.__class__.__name__ for metric in metrics_list],
                "pipeline_type": "haystack_custom_components",
                **(config or {})
            },
            reinit=True
        )
        
        print(f"🚀 Started W&B experiment: {self.experiment_name}")
        print(f"📊 View experiment at: {self.run.url}")
        
        # Initialize pipeline components
        reader = CSVReaderComponent()
        augmenter = RAGDataAugmenterComponent(rag_supercomponent=rag_supercomponent)
        evaluator = RagasEvaluationComponent(metrics=metrics_list)
        
        # Build the pipeline (simplified without WeaveConnector)
        self.pipeline = Pipeline()
        
        # Add components
        self.pipeline.add_component("reader", reader)
        self.pipeline.add_component("augmenter", augmenter)
        self.pipeline.add_component("evaluator", evaluator)
        
        # Connect components
        self.pipeline.connect("reader.data_frame", "augmenter.data_frame")
        self.pipeline.connect("augmenter.augmented_data_frame", "evaluator.augmented_data_frame")
        
        print("✅ Pipeline setup complete with W&B tracking enabled")
        
        return self.pipeline
    
    def run_evaluation(self, 
                      csv_file_path: str,
                      log_detailed_results: bool = True):
        """
        Execute the evaluation pipeline and log results to W&B.
        
        Args:
            csv_file_path: Path to the evaluation dataset CSV
            log_detailed_results: Whether to log detailed per-query results
            
        Returns:
            dict: Evaluation results with metrics and detailed dataframe
        """
        if not self.pipeline:
            raise ValueError("Pipeline not set up. Call setup_pipeline() first.")
        
        print(f"📈 Running evaluation on: {csv_file_path}")
        
        # Log the dataset as an artifact
        dataset_artifact = wandb.Artifact(
            name=f"evaluation-dataset-{Path(csv_file_path).stem}",
            type="dataset"
        )
        dataset_artifact.add_file(csv_file_path)
        self.run.log_artifact(dataset_artifact)
        
        # Execute the pipeline
        start_time = datetime.now()
        results = self.pipeline.run({"reader": {"source": csv_file_path}})
        end_time = datetime.now()
        
        execution_time = (end_time - start_time).total_seconds()
        
        # Extract metrics and detailed results
        metrics = results["evaluator"]["metrics"]  # This is an EvaluationResult object
        evaluation_df = results["evaluator"]["evaluation_df"]
        
        # Log summary metrics to W&B
        wandb_metrics = {
            "execution_time_seconds": execution_time,
            "num_queries_evaluated": len(evaluation_df),
        }
        
        # Handle RAGAS EvaluationResult object properly
        # Convert EvaluationResult to dictionary format
        try:
            # Try to get metrics as a dictionary
            if hasattr(metrics, 'to_dict'):
                metrics_dict = metrics.to_dict()
            elif hasattr(metrics, '__dict__'):
                metrics_dict = metrics.__dict__
            else:
                # If it's already a dict, use it directly
                metrics_dict = metrics if isinstance(metrics, dict) else {}
            
            # Add RAGAS metrics to W&B logging
            for metric_name, metric_value in metrics_dict.items():
                if isinstance(metric_value, (int, float)):
                    wandb_metrics[f"ragas_{metric_name}"] = metric_value
                elif hasattr(metric_value, 'value'):  # Some metrics might have a .value attribute
                    if isinstance(metric_value.value, (int, float)):
                        wandb_metrics[f"ragas_{metric_name}"] = metric_value.value
        except Exception as e:
            print(f"⚠️ Warning: Could not extract all metrics for W&B logging: {e}")
            print(f"Metrics type: {type(metrics)}")
        
        self.run.log(wandb_metrics)
        
        print(f"📊 Logged {len(wandb_metrics)} metrics to W&B")
        
        # Log detailed results if requested
        if log_detailed_results:
            # Create results artifact
            results_artifact = wandb.Artifact(
                name=f"evaluation-results-{self.experiment_name}",
                type="evaluation_results"
            )
            
            # Save detailed results to CSV
            results_file = f"detailed_results_{self.experiment_name}.csv"
            evaluation_df.to_csv(results_file, index=False)
            results_artifact.add_file(results_file)
            
            # Log the results artifact
            self.run.log_artifact(results_artifact)
            
            # Create a summary table for W&B dashboard
            summary_table = wandb.Table(dataframe=evaluation_df.head(10))  # Show first 10 rows
            self.run.log({"evaluation_sample_results": summary_table})
            
            print("📋 Logged detailed results and sample table to W&B")
        
        return {
            "metrics": metrics,
            "evaluation_df": evaluation_df,
            "execution_time": execution_time,
            "wandb_url": self.run.url
        }
    
    def finish_experiment(self):
        """Finish the W&B run and cleanup."""
        if self.run:
            self.run.finish()
            print(f"✅ Experiment {self.experiment_name} completed")
            print(f"🔗 View results at: {self.run.url}")

print("✅ RAGEvaluationExperiment class defined successfully")

✅ RAGEvaluationExperiment class defined successfully


## 🔬 Experiment 1: Naive RAG with W&B Tracking

Let's run our first experiment: evaluating the Naive RAG system with comprehensive W&B tracking.

In [4]:
# Define evaluation metrics
evaluation_metrics = [
    LLMContextRecall(), 
    Faithfulness(), 
    FactualCorrectness(), 
    ResponseRelevancy(), 
    ContextEntityRecall(), 
    NoiseSensitivity()
]

# Configuration for the experiment
naive_rag_config = {
    "embedder_model": "sentence-transformers/all-MiniLM-L6-v2",
    "llm_model": "gpt-4o-mini",
    "retriever_top_k": 3,
    "rag_type": "naive",
    "document_store": "elasticsearch"
}

# Initialize experiment
naive_experiment = RAGEvaluationExperiment(
    project_name="haystack-rag-evaluation",
    experiment_name="naive-rag-baseline"
)

# Setup pipeline
pipeline = naive_experiment.setup_pipeline(
    rag_supercomponent=naive_rag_sc,
    metrics_list=evaluation_metrics,
    config=naive_rag_config
)

print("🔧 Naive RAG experiment setup complete")

wandb: Currently logged in as: lgutierrwr to https://api.wandb.ai. Use `wandb login --relogin` to force relogin
wandb: WARNING Using a boolean value for 'reinit' is deprecated. Use 'return_previous' or 'finish_previous' instead.
wandb: WARNING Using a boolean value for 'reinit' is deprecated. Use 'return_previous' or 'finish_previous' instead.


wandb: Initializing weave.
weave: Logged in as Weights & Biases user: lgutierrwr.
weave: View Weave data at https://wandb.ai/lgutierrwr/haystack-rag-evaluation/weave
weave: Logged in as Weights & Biases user: lgutierrwr.
weave: View Weave data at https://wandb.ai/lgutierrwr/haystack-rag-evaluation/weave


🚀 Started W&B experiment: naive-rag-baseline
📊 View experiment at: https://wandb.ai/lgutierrwr/haystack-rag-evaluation/runs/g20hks5o
✅ Pipeline setup complete with W&B tracking enabled
🔧 Naive RAG experiment setup complete


In [ ]:
# Run the evaluation
csv_file_path = "data_for_eval/synthetic_tests_advanced_branching_10.csv"

naive_results = naive_experiment.run_evaluation(
    csv_file_path=csv_file_path,
    log_detailed_results=True
)

print("\n📊 Naive RAG Results Summary:")
print(f"⏱️  Execution time: {naive_results['execution_time']:.2f} seconds")
print(f"🔗 W&B Dashboard: {naive_results['wandb_url']}")


📈 Running evaluation on: data_for_eval/synthetic_tests_advanced_branching_10.csv
Loaded DataFrame with 10 rows from data_for_eval/synthetic_tests_advanced_branching_10.csv.
Running RAG SuperComponent on 10 queries...
Loaded DataFrame with 10 rows from data_for_eval/synthetic_tests_advanced_branching_10.csv.
Running RAG SuperComponent on 10 queries...


Batches: 100%|██████████| 1/1 [00:00<00:00,  7.99it/s]
weave: 🍩 https://wandb.ai/lgutierrwr/haystack-rag-evaluation/r/call/019a42f8-6996-77e6-96b2-fc328d67c3ab

weave: 🍩 https://wandb.ai/lgutierrwr/haystack-rag-evaluation/r/call/019a42f8-6996-77e6-96b2-fc328d67c3ab
Batches: 100%|██████████| 1/1 [00:00<00:00,  6.86it/s]
weave: 🍩 https://wandb.ai/lgutierrwr/haystack-rag-evaluation/r/call/019a42f8-7825-75f7-8000-845b407e9596

weave: 🍩 https://wandb.ai/lgutierrwr/haystack-rag-evaluation/r/call/019a42f8-7825-75f7-8000-845b407e9596
Batches: 100%|██████████| 1/1 [00:00<00:00,  8.36it/s]
weave: 🍩 https://wandb.ai/lgutierrwr/haystack-rag-evaluation/r/call/019a42f8-80a1-7536-a722-33ddaef3f4af

weave: 🍩 https://wandb.ai/lgutierrwr/haystack-rag-evaluation/r/call/019a42f8-80a1-7536-a722-33ddaef3f4af
Batches: 100%|██████████| 1/1 [00:00<00:00, 13.56it/s]
weave: 🍩 https://wandb.ai/lgutierrwr/haystack-rag-evaluation/r/call/019a42f8-9d51-7ea9-a83c-05f74993dd63

weave: 🍩 https://wandb.ai/lgutierrwr/hays

RAG processing complete.
Creating Ragas EvaluationDataset...
Starting Ragas evaluation...


Evaluating:   0%|          | 0/60 [00:00<?, ?it/s]weave: 🍩 https://wandb.ai/lgutierrwr/haystack-rag-evaluation/r/call/019a42f9-082d-7922-9a53-f88990f10730
weave: 🍩 https://wandb.ai/lgutierrwr/haystack-rag-evaluation/r/call/019a42f9-082f-7a55-8830-851052e98f94
weave: 🍩 https://wandb.ai/lgutierrwr/haystack-rag-evaluation/r/call/019a42f9-0830-7f3a-8a52-14433f8915f9
weave: 🍩 https://wandb.ai/lgutierrwr/haystack-rag-evaluation/r/call/019a42f9-0832-71c0-882b-b67744cebfc8
weave: 🍩 https://wandb.ai/lgutierrwr/haystack-rag-evaluation/r/call/019a42f9-082d-7922-9a53-f88990f10730
weave: 🍩 https://wandb.ai/lgutierrwr/haystack-rag-evaluation/r/call/019a42f9-082f-7a55-8830-851052e98f94
weave: 🍩 https://wandb.ai/lgutierrwr/haystack-rag-evaluation/r/call/019a42f9-0830-7f3a-8a52-14433f8915f9
weave: 🍩 https://wandb.ai/lgutierrwr/haystack-rag-evaluation/r/call/019a42f9-0832-71c0-882b-b67744cebfc8
weave: 🍩 https://wandb.ai/lgutierrwr/haystack-rag-evaluation/r/call/019a42f9-0835-765f-9c55-e926684a5c3d
weave

Ragas evaluation complete.
Overall metrics: {'context_recall': 1.0000, 'faithfulness': 0.6587, 'factual_correctness(mode=f1)': 0.4630, 'answer_relevancy': 0.5767, 'context_entity_recall': 0.2030, 'noise_sensitivity(mode=relevant)': 0.1559}
📊 Logged 2 metrics to W&B
📋 Logged detailed results and sample table to W&B

📊 Naive RAG Results Summary:
⏱️  Execution time: 250.77 seconds
🔗 W&B Dashboard: https://wandb.ai/lgutierrwr/haystack-rag-evaluation/runs/g20hks5o

📈 RAGAS Metrics:
📋 Logged detailed results and sample table to W&B

📊 Naive RAG Results Summary:
⏱️  Execution time: 250.77 seconds
🔗 W&B Dashboard: https://wandb.ai/lgutierrwr/haystack-rag-evaluation/runs/g20hks5o

📈 RAGAS Metrics:


AttributeError: 'EvaluationResult' object has no attribute 'items'

In [15]:
naive_results['metrics']

{'context_recall': 1.0000, 'faithfulness': 0.6587, 'factual_correctness(mode=f1)': 0.4630, 'answer_relevancy': 0.5767, 'context_entity_recall': 0.2030, 'noise_sensitivity(mode=relevant)': 0.1559}

In [17]:
naive_results['wandb_url']

'https://wandb.ai/lgutierrwr/haystack-rag-evaluation/runs/g20hks5o'

In [6]:
# Finish the naive RAG experiment
naive_experiment.finish_experiment()

execution_time_seconds,▁
num_queries_evaluated,▁
execution_time_seconds,250.76971
num_queries_evaluated,10


✅ Experiment naive-rag-baseline completed
🔗 View results at: https://wandb.ai/lgutierrwr/haystack-rag-evaluation/runs/g20hks5o


In [13]:
naive_results['execution_time']

250.769709

In [8]:
print("\n📈 RAGAS Metrics:")
naive_results['metrics']


📈 RAGAS Metrics:


{'context_recall': 1.0000, 'faithfulness': 0.6587, 'factual_correctness(mode=f1)': 0.4630, 'answer_relevancy': 0.5767, 'context_entity_recall': 0.2030, 'noise_sensitivity(mode=relevant)': 0.1559}

In [10]:
naive_results['evaluation_df']

,user_input,retrieved_contexts,reference_contexts,response,reference,context_recall,faithfulness,factual_correctness(mode=f1),answer_relevancy,context_entity_recall,noise_sensitivity(mode=relevant)
0,What are the ethical implications and concerns...,"[What is AI, how does it work and why are some...","[What is AI, how does it work and why are some...",The ethical implications and concerns surround...,"The rise of Meta AI, like other generative AI ...",1.0,1.000000,0.80,0.990249,0.050000,0.250000
1,What is the estimated energy consumption of th...,[NBER WORKING PAPER SERIES\nHOW PEOPLE USE CHA...,[How does AI effect the environment?\nIt is no...,Some researchers estimate that the AI industry...,Some researchers estimate that the AI industry...,1.0,1.000000,1.00,0.000000,0.000000,0.000000
2,Wut is the significanse of Artificial Intellig...,"[What is AI, how does it work and why are some...",[This article was published in 2018. To read m...,Artificial Intelligence (AI) plays a significa...,Artificial Intelligence (AI) is a technology t...,1.0,NaN,0.24,0.964897,0.333333,NaN
3,What does Figure 22 illustrate about the varia...,[•Sampled from all ChatGPT users:a random samp...,[<1-hop>\n\n37% of messages are work-related\n...,I don't have enough information to answer.,Figure 22 illustrates the variation in ChatGPT...,1.0,0.000000,0.00,0.000000,0.100000,0.000000
4,What does Figure 22 show about how ChatGPT is ...,[•Sampled from all ChatGPT users:a random samp...,[<1-hop>\n\nPanel A.Work Related\n Panel B1.As...,I don't have enough information to answer.,Figure 22 illustrates the classification of wo...,1.0,0.000000,0.00,0.000000,0.437500,0.000000
5,How does ChatGPT Business usage vary by occupa...,[•Sampled from all ChatGPT users:a random samp...,[<1-hop>\n\nCorporate users may also use ChatG...,I don't have enough information to answer.,ChatGPT Business usage varies significantly by...,1.0,0.000000,0.00,0.000000,0.100000,0.000000
6,How does the environmental impact of artificia...,"[What is AI, how does it work and why are some...","[<1-hop>\n\nWhat is AI, how does it work and w...",The environmental impact of artificial intelli...,The environmental impact of artificial intelli...,1.0,1.000000,0.71,0.949340,0.105263,0.133333
7,How do privacy protections and de-identificati...,[•Sampled from all ChatGPT users:a random samp...,[<1-hop>\n\nWe describe the contents of each d...,The analysis of ChatGPT user messages employs ...,Privacy protections in the analysis of ChatGPT...,1.0,1.000000,0.41,0.952696,0.333333,0.590909
8,What trends can be observed in user cohort ana...,[•Sampled from all ChatGPT users:a random samp...,[<1-hop>\n\nThe yellow line represents the fir...,User cohort analysis reveals significant trend...,User cohort analysis reveals that there has be...,1.0,1.000000,0.80,0.956741,0.153846,NaN
9,What are the environmental concerns related to...,"[What is AI, how does it work and why are some...","[<1-hop>\n\nWhat is AI, how does it work and w...",The environmental concerns related to artifici...,The environmental concerns related to artifici...,1.0,0.928571,0.67,0.953338,0.416667,0.272727


## 🚀 Experiment 2: Hybrid RAG with W&B Tracking

Now let's run a comparative experiment with the Hybrid RAG system to see performance differences.

In [ ]:
# Configuration for hybrid RAG experiment
hybrid_rag_config = {
    "embedder_model": "sentence-transformers/all-MiniLM-L6-v2",
    "llm_model": "gpt-4o-mini",
    "retriever_top_k": 3,
    "rag_type": "hybrid",
    "document_store": "elasticsearch",
    "bm25_enabled": True,
    "reranker_model": "BAAI/bge-reranker-base",
    "dense_retrieval": True,
    "sparse_retrieval": True
}

# Initialize hybrid experiment
hybrid_experiment = RAGEvaluationExperiment(
    project_name="haystack-rag-evaluation",
    experiment_name="hybrid-rag-enhanced"
)

# Setup pipeline
hybrid_pipeline = hybrid_experiment.setup_pipeline(
    rag_supercomponent=hybrid_rag_sc,
    metrics_list=evaluation_metrics,
    config=hybrid_rag_config
)

print("🔧 Hybrid RAG experiment setup complete")

In [ ]:
# Run the hybrid evaluation
hybrid_results = hybrid_experiment.run_evaluation(
    csv_file_path=csv_file_path,
    log_detailed_results=True
)

print("\n📊 Hybrid RAG Results Summary:")
print(f"⏱️  Execution time: {hybrid_results['execution_time']:.2f} seconds")
print(f"🔗 W&B Dashboard: {hybrid_results['wandb_url']}")
print("\n📈 RAGAS Metrics:")
for metric, value in hybrid_results['metrics'].items():
    if isinstance(value, (int, float)):
        print(f"  {metric}: {value:.4f}")

In [ ]:
# Finish the hybrid RAG experiment
hybrid_experiment.finish_experiment()

## 📊 Comparative Analysis with W&B

Let's create a simple comparison of the results and demonstrate how to analyze them programmatically.

In [ ]:
# Create comparison summary
def create_comparison_summary(naive_results, hybrid_results):
    """
    Create a comparison summary between two RAG system evaluations.
    """
    
    comparison = {
        "System": ["Naive RAG", "Hybrid RAG"],
        "Execution Time (s)": [
            naive_results['execution_time'], 
            hybrid_results['execution_time']
        ]
    }
    
    # Compare metrics
    for metric_name in naive_results['metrics'].keys():
        if isinstance(naive_results['metrics'][metric_name], (int, float)):
            comparison[metric_name] = [
                naive_results['metrics'][metric_name],
                hybrid_results['metrics'][metric_name]
            ]
    
    return pd.DataFrame(comparison)

# Create and display comparison
comparison_df = create_comparison_summary(naive_results, hybrid_results)
print("🔍 RAG Systems Comparison:")
print(comparison_df.to_string(index=False, float_format='%.4f'))

# Calculate improvements
print("\n📈 Performance Improvements (Hybrid vs Naive):")
for col in comparison_df.columns[2:]:  # Skip System and Execution Time columns
    if col in comparison_df.columns:
        naive_val = comparison_df.loc[0, col]
        hybrid_val = comparison_df.loc[1, col]
        if isinstance(naive_val, (int, float)) and isinstance(hybrid_val, (int, float)):
            improvement = ((hybrid_val - naive_val) / naive_val) * 100
            print(f"  {col}: {improvement:+.2f}%")

## 🎯 Advanced W&B Features: Sweeps and Hyperparameter Optimization

Let's demonstrate how to set up a W&B sweep for systematic hyperparameter optimization of RAG systems.

In [ ]:
def create_sweep_config():
    """
    Create a W&B sweep configuration for RAG hyperparameter optimization.
    """
    
    sweep_config = {
        'method': 'bayes',  # Can be 'grid', 'random', or 'bayes'
        'metric': {
            'name': 'ragas_faithfulness',
            'goal': 'maximize'
        },
        'parameters': {
            'retriever_top_k': {
                'values': [3, 5, 7, 10]
            },
            'rag_system': {
                'values': ['naive', 'hybrid']
            },
            'dataset_size': {
                'values': ['small', 'medium', 'large']  # Different CSV files
            }
        }
    }
    
    return sweep_config

def sweep_function():
    """
    Function to run a single sweep iteration.
    This would be called by W&B sweep agent.
    """
    
    # Initialize W&B run for sweep
    run = wandb.init()
    
    # Get sweep parameters
    config = wandb.config
    
    # Select RAG system based on sweep parameter
    if config.rag_system == 'naive':
        rag_sc = naive_rag_sc
    else:
        rag_sc = hybrid_rag_sc
    
    # Select dataset based on sweep parameter
    dataset_map = {
        'small': 'data_for_eval/synthetic_tests_advanced_branching_3.csv',
        'medium': 'data_for_eval/synthetic_tests_advanced_branching_10.csv',
        'large': 'data_for_eval/synthetic_tests_advanced_branching_50.csv'
    }
    
    dataset_path = dataset_map[config.dataset_size]
    
    # Create experiment with sweep configuration
    experiment = RAGEvaluationExperiment(
        project_name="rag-hyperparameter-sweep",
        experiment_name=f"sweep-{config.rag_system}-{config.dataset_size}-k{config.retriever_top_k}"
    )
    
    # Note: In a real sweep, you'd modify the RAG components based on config.retriever_top_k
    # For this demo, we'll use the existing components
    
    pipeline = experiment.setup_pipeline(
        rag_supercomponent=rag_sc,
        metrics_list=evaluation_metrics,
        config=dict(config)
    )
    
    # Run evaluation
    results = experiment.run_evaluation(
        csv_file_path=dataset_path,
        log_detailed_results=False  # Skip detailed logging for sweeps
    )
    
    experiment.finish_experiment()

# Display sweep configuration
sweep_config = create_sweep_config()
print("🔍 W&B Sweep Configuration:")
print(json.dumps(sweep_config, indent=2))

print("\n💡 To run this sweep:")
print("1. Create the sweep: sweep_id = wandb.sweep(sweep_config, project='rag-hyperparameter-sweep')")
print("2. Run the agent: wandb.agent(sweep_id, sweep_function)")

## 📚 W&B Dashboard Guide

Here's how to make the most of your W&B dashboard for RAG evaluation analysis:

### 🎛️ Key Dashboard Features

**1. Experiment Comparison**
- Navigate to your project: `https://wandb.ai/your-username/haystack-rag-evaluation`
- Use the "Compare" feature to view metrics side-by-side
- Create custom charts for metric trends over time

**2. Metric Visualization**
- **Parallel Coordinates Plot**: Compare multiple metrics simultaneously
- **Scatter Plots**: Identify correlations between different metrics
- **Bar Charts**: Compare performance across different RAG systems

**3. Artifact Management**
- **Datasets**: Version control your evaluation datasets
- **Results**: Store and compare detailed evaluation results
- **Models**: Track different RAG configurations

**4. Pipeline Tracing with Weave**
- View detailed execution traces for each pipeline run
- Identify bottlenecks and performance issues
- Debug component interactions and data flow

### 🔧 Custom Dashboard Setup

**Recommended Visualizations:**
1. **RAG Performance Overview**: Line chart showing all RAGAS metrics over time
2. **System Comparison**: Bar chart comparing Naive vs Hybrid RAG performance
3. **Execution Time Analysis**: Scatter plot of metrics vs execution time
4. **Query Difficulty Analysis**: Heatmap showing performance per query type

### 📊 Team Collaboration Features

- **Shared Projects**: Collaborate with team members on RAG development
- **Comments & Annotations**: Add insights directly to experiment runs
- **Reports**: Create comprehensive evaluation reports with embedded visualizations
- **Alerts**: Set up notifications for performance thresholds

## 🎓 Summary: RAG Evaluation with Experiment Tracking

### ✅ What You've Accomplished

Congratulations! You've successfully integrated **Weights & Biases experiment tracking** with your RAG evaluation pipeline. Here's what you've learned:

**🔧 Technical Integration:**
- Set up W&B WeaveConnector for pipeline tracing
- Created comprehensive experiment tracking workflow
- Integrated RAGAS metrics with W&B logging
- Implemented artifact management for datasets and results

**📊 Experiment Management:**
- Systematic comparison of different RAG systems
- Reproducible evaluation workflows with version control
- Comprehensive metric tracking and visualization
- Team collaboration through shared dashboards

**🚀 Advanced Capabilities:**
- Hyperparameter optimization with W&B sweeps
- Pipeline performance monitoring and debugging
- Automated experiment organization and comparison
- Rich visualization and reporting capabilities

### 🎯 Key Benefits Achieved

1. **🔄 Reproducibility**: Every experiment is tracked with complete configuration
2. **📈 Scalability**: Easy to run large-scale evaluation campaigns
3. **🤝 Collaboration**: Share insights and results with team members
4. **🎯 Optimization**: Systematic hyperparameter tuning and performance improvement
5. **📊 Insights**: Rich visualizations reveal patterns and optimization opportunities

### 🚀 Next Steps

**Immediate Applications:**
1. **Scale Experiments**: Run evaluations on larger, diverse datasets
2. **Hyperparameter Sweeps**: Optimize RAG configurations systematically
3. **A/B Testing**: Compare different embedding models, chunk sizes, retrieval strategies
4. **Production Monitoring**: Deploy evaluation pipeline for continuous monitoring

**Advanced Extensions:**
1. **Custom Metrics**: Integrate domain-specific evaluation metrics
2. **Real-time Dashboards**: Monitor RAG performance in production
3. **Automated Alerts**: Set up performance threshold notifications
4. **Multi-modal RAG**: Extend evaluation to text, image, and multimodal systems

---

**🎉 Congratulations!** You now have a **production-ready, experiment-tracked RAG evaluation system**. This workflow will serve as the foundation for systematic RAG development, optimization, and team collaboration.

**🔗 Resources:**
- [W&B Documentation](https://docs.wandb.ai/)
- [Haystack W&B Integration](https://docs.haystack.deepset.ai/reference/integrations-weights-bias)
- [RAGAS Documentation](https://docs.ragas.io/)
- [Your W&B Project Dashboard](https://wandb.ai/your-username/haystack-rag-evaluation)